In [1]:
import sys
import os

sys.path.insert(1, os.path.join(sys.path[0], '..')) 

In [2]:
import numpy as np
import pandas as pd
import yaml
from pathlib import Path

import simulation
import stats

In [3]:
simulation_config_name = "simulation_config.yml"
simulation_config_folder = Path('../config')
simulation_config_path = simulation_config_folder / simulation_config_name

In [4]:
with open(simulation_config_path, "r") as f:
    cfg_load = yaml.safe_load(f)

In [5]:
cfg = cfg_load['simulation_config']
premo_cfg = cfg['premo']

In [6]:
np.random.seed(2023)

In [11]:
RANDOM_SEED = cfg['random_seed'] 
N_TRIALS = premo_cfg['n_trials']
N_CARDS = premo_cfg['n_cards']
MAX_RIFFLE_SHUFFLES = premo_cfg['max_riffle_shuffles']

In [5]:
results = simulation.premo_simulation(
    n_trials = N_TRIALS, 
    n_cards_in_deck=N_CARDS, 
    max_riffle_shuffle=MAX_RIFFLE_SHUFFLES
    )

In [6]:
# Calculate winding distances for each card. We wish to create a data set with the winding distances per card, 
# per trial, per shuffle.
# Later we will use this data to create a big matrix with all the premo information needed to perform the guesses

data = []

for trial in results:
    d = trial['deck']
    row = {card: stats.winding_distance(d, card) for card in d}
    row['top_card'] = trial['top_card']
    row['trial'] = trial['trial']
    row['shuffle'] = trial['shuffle']
    data.append(row)

In [7]:
premo_df = pd.DataFrame(data)
premo_df.set_index(['trial', 'shuffle'], inplace=True)

In [8]:
premo_df.head()

9  35  10  36  37  24  38  39  40  11  ...  30  31   5   6  \
trial shuffle                                          ...                   
1     1         4   3   8   2   2  53   2   1   3   7  ...   2   3   3   2   
      2         8   4  12   6   4  31   1   2   5  11  ...   5   8   7   6   
      3        16   8  16   9   5   4   1   2  10  16  ...  11  32  26  16   
      4        34  21  39  18  13   7   9  10  18  34  ...  24  16  10  32   
      5        70  48  29  64  75  17  45  19  32  15  ...  47  39  18  20   

               32   7   8  33  34  top_card  
trial shuffle                                
1     1         5   2   3   3   2        24  
      2        12   5   7   8   4        23  
      3        38  15  16  17  10        15  
      4        54  29  29  83  44        33  
      5        54  11  50  58  40        37  

[5 rows x 53 columns]

In [9]:
# Create a large Matrix that holds the guesses, with the highest winding distances and the top_card

cols = premo_df.columns.to_list().pop(-1)

guesses_df = pd.DataFrame(index=premo_df.index)
guesses_cols = []
for g in range(1, MAX_GUESSES + 1):
    col_name = f"guess_{g}"
    guesses_df[col_name] = premo_df.drop(columns=['top_card']).apply(lambda row: row.nlargest(g).index.tolist(), axis=1)
    df = premo_df.join(guesses_df)
    guesses_cols.append(col_name)
    
df = df[guesses_cols + ['top_card']]

In [10]:
df.head()

guess_1   guess_2       guess_3           guess_4  \
trial shuffle                                                     
1     1          [24]  [24, 25]  [24, 25, 23]  [24, 25, 23, 10]   
      2          [23]  [23, 24]  [23, 24, 22]  [23, 24, 22, 52]   
      3          [15]  [15, 16]  [15, 16, 14]  [15, 16, 14, 32]   
      4          [33]  [33, 48]  [33, 48, 47]  [33, 48, 47, 32]   
      5          [47]  [47, 48]  [47, 48, 49]  [47, 48, 49, 37]   

                            guess_5                   guess_6  \
trial shuffle                                                   
1     1        [24, 25, 23, 10, 11]  [24, 25, 23, 10, 11, 48]   
      2        [23, 24, 22, 52, 10]  [23, 24, 22, 52, 10, 48]   
      3        [15, 16, 14, 32, 48]   [15, 16, 14, 32, 48, 4]   
      4        [33, 48, 47, 32, 34]  [33, 48, 47, 32, 34, 49]   
      5        [47, 48, 49, 37, 27]   [47, 48, 49, 37, 27, 9]   

                                    guess_7                           guess_8  \
trial shuffle                                                                   
1     1        [24, 25, 23, 10, 11, 48, 16]  [24, 25, 23, 10, 11, 48, 16, 17]   
      2         [23, 24, 22, 52, 10, 48, 4]   [23, 24, 22, 52, 10, 48, 4, 32]   
      3         [15, 16, 14, 32, 48, 4, 47]   [15, 16, 14, 32, 48, 4, 47, 17]   
      4        [33, 48, 47, 32, 34, 49, 52]  [33, 48, 47, 32, 34, 49, 52, 10]   
      5         [47, 48, 49, 37, 27, 9, 46]   [47, 48, 49, 37, 27, 9, 46, 19]   

                                           guess_9  \
trial shuffle                                        
1     1        [24, 25, 23, 10, 11, 48, 16, 17, 4]   
      2        [23, 24, 22, 52, 10, 48, 4, 32, 11]   
      3        [15, 16, 14, 32, 48, 4, 47, 17, 31]   
      4        [33, 48, 47, 32, 34, 49, 52, 10, 1]   
      5        [47, 48, 49, 37, 27, 9, 46, 19, 50]   

                                              guess_10  top_card  
trial shuffle                                                     
1     1        [24, 25, 23, 10, 11, 48, 16, 17, 4, 32]        24  
      2        [23, 24, 22, 52, 10, 48, 4, 32, 11, 43]        23  
      3         [15, 16, 14, 32, 48, 4, 47, 17, 31, 5]        15  
      4        [33, 48, 47, 32, 34, 49, 52, 10, 1, 26]        33  
      5        [47, 48, 49, 37, 27, 9, 46, 19, 50, 36]        37

In [11]:
# Now check per each guess number, if the top_card is among the guesses
correct_cols = []

for guess_col_name in guesses_cols:
    correct_col_name = guess_col_name + "_correct"
    df[correct_col_name] = [row['top_card'] in row[guess_col_name] for _, row in df.iterrows()]
    correct_cols.append(correct_col_name)

In [12]:
df.head()

guess_1   guess_2       guess_3           guess_4  \
trial shuffle                                                     
1     1          [24]  [24, 25]  [24, 25, 23]  [24, 25, 23, 10]   
      2          [23]  [23, 24]  [23, 24, 22]  [23, 24, 22, 52]   
      3          [15]  [15, 16]  [15, 16, 14]  [15, 16, 14, 32]   
      4          [33]  [33, 48]  [33, 48, 47]  [33, 48, 47, 32]   
      5          [47]  [47, 48]  [47, 48, 49]  [47, 48, 49, 37]   

                            guess_5                   guess_6  \
trial shuffle                                                   
1     1        [24, 25, 23, 10, 11]  [24, 25, 23, 10, 11, 48]   
      2        [23, 24, 22, 52, 10]  [23, 24, 22, 52, 10, 48]   
      3        [15, 16, 14, 32, 48]   [15, 16, 14, 32, 48, 4]   
      4        [33, 48, 47, 32, 34]  [33, 48, 47, 32, 34, 49]   
      5        [47, 48, 49, 37, 27]   [47, 48, 49, 37, 27, 9]   

                                    guess_7                           guess_8  \
trial shuffle                                                                   
1     1        [24, 25, 23, 10, 11, 48, 16]  [24, 25, 23, 10, 11, 48, 16, 17]   
      2         [23, 24, 22, 52, 10, 48, 4]   [23, 24, 22, 52, 10, 48, 4, 32]   
      3         [15, 16, 14, 32, 48, 4, 47]   [15, 16, 14, 32, 48, 4, 47, 17]   
      4        [33, 48, 47, 32, 34, 49, 52]  [33, 48, 47, 32, 34, 49, 52, 10]   
      5         [47, 48, 49, 37, 27, 9, 46]   [47, 48, 49, 37, 27, 9, 46, 19]   

                                           guess_9  \
trial shuffle                                        
1     1        [24, 25, 23, 10, 11, 48, 16, 17, 4]   
      2        [23, 24, 22, 52, 10, 48, 4, 32, 11]   
      3        [15, 16, 14, 32, 48, 4, 47, 17, 31]   
      4        [33, 48, 47, 32, 34, 49, 52, 10, 1]   
      5        [47, 48, 49, 37, 27, 9, 46, 19, 50]   

                                              guess_10  ...  guess_1_correct  \
trial shuffle                                           ...                    
1     1        [24, 25, 23, 10, 11, 48, 16, 17, 4, 32]  ...             True   
      2        [23, 24, 22, 52, 10, 48, 4, 32, 11, 43]  ...             True   
      3         [15, 16, 14, 32, 48, 4, 47, 17, 31, 5]  ...             True   
      4        [33, 48, 47, 32, 34, 49, 52, 10, 1, 26]  ...             True   
      5        [47, 48, 49, 37, 27, 9, 46, 19, 50, 36]  ...            False   

               guess_2_correct  guess_3_correct  guess_4_correct  \
trial shuffle                                                      
1     1                   True             True             True   
      2                   True             True             True   
      3                   True             True             True   
      4                   True             True             True   
      5                  False            False             True   

               guess_5_correct  guess_6_correct  guess_7_correct  \
trial shuffle                                                      
1     1                   True             True             True   
      2                   True             True             True   
      3                   True             True             True   
      4                   True             True             True   
      5                   True             True             True   

               guess_8_correct  guess_9_correct  guess_10_correct  
trial shuffle                                                      
1     1                   True             True              True  
      2                   True             True              True  
      3                   True             True              True  
      4                   True             True              True  
      5                   True             True              True  

[5 rows x 21 columns]

In [33]:
# df now holds all the correct guesses, per number of guesses. This allows us to do analyses on the premo trick:
# i.e. See how many correct cards are guessed after 3 shuffles and 1 guess:

shuffle_num = 3
df.xs(shuffle_num, level='shuffle', axis=0).guess_1_correct.value_counts(normalize=True) 

True     0.84084
False    0.15916
Name: guess_1_correct, dtype: float64